The main solving big ridge regression problems

In [6]:
workspace()

LoadError: Main can only be replaced from the top level

In [7]:
using Plots

In [8]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [26]:
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [ ]:
dataset = "rcv1_train.multiclass"   # news20.binary  rcv1_train.multiclass rcv1_train.binary mnist.scale  a9a  gisette_scale 
prob = defineProb(dataset);

In [11]:
norm(prob.A*prob.xsol - prob.b)
showDetails(dataset)

LoadError: KeyError: key "mnist.scale" not found

In [19]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [20]:
options.AUX = [0 2]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 100; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 50; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 5; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 50]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 2; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-100-2
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 1.42  |  1.03  | 0.2253 
 50  | 0.44  |  0.42  | 0.3611 
 75  | 0.18  |  0.20  | 0.5063 
100  | 0.07  |  0.12  | 0.6361 
125  | 0.04  |  0.07  | 0.7538 
150  | 0.03  |  0.05  | 0.9674 
175  | 0.01  |  0.03  | 1.1544 
200  | 0.01  |  0.02  | 1.2695 
225  | 0.01  |  0.01  | 1.3760 
250  | 0.05  |  0.01  | 1.4873 
275  | 0.00  |  0.01  | 1.5897 
300  | 0.00  |  0.01  | 1.7086 
325  | 0.00  |  0.00  | 1.8274 
350  | 0.00  |  0.00  | 1.9400 
375  | 0.00  |  0.00  | 2.0529 
400  | 0.00  |  0.00  | 2.1636 
425  | 0.00  |  0.00  | 2.2946 
450  | 0.00  |  0.00  | 2.4160 
475  | 0.00  |  0.00  | 2.8095 
500  | 0.00  |  0.00  | 2.9365 
525  | 0.00  |  0.00  | 3.0516 
550  | 0.00  |  0.00  | 3.1741 
  3.247575 seconds (1.46 M allocations: 1.216 GB, 6.58% gc time)
rademacher-50-5
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 5.95  |  2.98  | 0.0687 
 50  | 2.87  |

1550  | 4.70  |  2.40  | 0.5816 
1575  | 4.64  |  2.01  | 0.5915 
1600  | 4.62  |  2.30  | 0.6015 
1625  | 4.57  |  2.79  | 0.6114 
1650  | 4.51  |  2.11  | 0.6194 
1675  | 4.46  |  1.95  | 0.6292 
1700  | 4.40  |  1.93  | 0.6391 
1725  | 4.34  |  2.18  | 0.6471 
1750  | 4.27  |  2.16  | 0.6568 
1775  | 4.22  |  2.08  | 0.6666 
1800  | 4.17  |  2.22  | 0.6766 
1825  | 4.14  |  2.01  | 0.6845 
1850  | 4.10  |  2.30  | 0.6945 
1875  | 4.03  |  1.89  | 0.7044 
1900  | 3.98  |  1.80  | 0.7124 
1925  | 3.95  |  1.86  | 0.7221 
1950  | 3.91  |  2.14  | 0.7319 
1975  | 3.86  |  1.85  | 0.7418 
2000  | 3.82  |  1.96  | 0.7497 
  0.833494 seconds (185.56 k allocations: 1.223 GB, 11.94% gc time)


In [ ]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =10; # Int(floor(log2(prob.n)));
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing countmin 
# options.sketchsize =10;#floor((prob.n)^(1/2));
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# @time output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [21]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 1.21  |  1.22  | 0.0045 
 50  | 0.41  |  0.46  | 0.0098 
 75  | 0.25  |  0.36  | 0.0149 
100  | 0.18  |  0.17  | 0.0198 
125  | 0.15  |  0.18  | 0.0324 
150  | 0.12  |  0.13  | 0.0365 
175  | 0.11  |  0.10  | 0.0405 
200  | 0.10  |  0.09  | 0.0472 
225  | 0.09  |  0.07  | 0.0520 
250  | 0.08  |  0.07  | 0.0565 
275  | 0.07  |  0.05  | 0.0602 
300  | 0.09  |  0.10  | 0.0828 
325  | 0.06  |  0.05  | 0.0899 
350  | 0.06  |  0.06  | 0.0926 
375  | 0.05  |  0.10  | 0.0954 
400  | 0.05  |  0.08  | 0.0980 
425  | 0.05  |  0.05  | 0.1080 
450  | 0.04  |  0.03  | 0.1197 
475  | 0.04  |  0.06  | 0.1221 
500  | 0.04  |  0.05  | 0.1245 
525  | 0.04  |  0.04  | 0.1269 
550  | 0.03  |  0.06  | 0.1326 
575  | 0.03  |  0.03  | 0.1371 
600  | 0.03  |  0.03  | 0.1395 
625  | 0.03  |  0.14  | 0.1418 
650  | 0.03  |  0.05  | 0.1442 
675  | 0.02  |  0.04  | 0.1466 
700  | 0.02  |  0.04  | 0.1490 
725  | 0.02  |  0.02  | 0

In [22]:
# Testing CD 
options.maxiter=5000;
options.skip_error_calculation = 250;
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
method_name = "CD";# gradient descent
options.sketchsize = 100;
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-10
-------------------
It   | Error% | Residual |  Time   
-------------------
250  | 3.45  |  2.42  | 0.0247 
500  | 1.50  |  1.51  | 0.0417 
750  | 0.87  |  0.92  | 0.0609 
1000  | 0.48  |  0.63  | 0.0786 
1250  | 0.32  |  0.48  | 0.0953 
1500  | 0.22  |  0.41  | 0.1081 
1750  | 0.15  |  0.32  | 0.1241 
2000  | 0.11  |  0.27  | 0.1399 
2250  | 0.07  |  0.26  | 0.1526 
2500  | 0.06  |  0.19  | 0.1689 
2750  | 0.04  |  0.17  | 0.1849 
3000  | 0.03  |  0.13  | 0.2008 
3250  | 0.03  |  0.12  | 0.2135 
3500  | 0.02  |  0.11  | 0.2293 
3750  | 0.08  |  0.07  | 0.2450 
4000  | 0.01  |  0.07  | 0.2608 
4250  | 0.01  |  0.06  | 0.2731 
4500  | 0.08  |  0.05  | 0.2881 
4750  | 0.00  |  0.05  | 0.3034 
5000  | 0.00  |  0.04  | 0.3180 
  0.379869 seconds (204.28 k allocations: 319.843 MB, 11.67% gc time)
CD-27
-------------------
It   | Error% | Residual |  Time   
-------------------
250  | 1.59  |  1.63  | 0.0441 
500  | 0.43  |  0.56  | 0.1142 
750  | 0.16  |  0.39  | 0.1673 
1000  | 0.06 

In [23]:
# OUTPUTS =[];
# include("../src/call_all_methods.jl")

In [24]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [25]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

mnist.scale


In [344]:
# Loading saved problem data
probname = "gisette_scale"#"gisette_scale";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];